In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ['OMP_NUM_THREADS']='32'
#os.environ['MKL_NUM_THREADS']='32'
from torch import autograd
from fastai.transforms import TfmType
from fasterai.transforms import *
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#torch.set_num_threads(32)

In [4]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'super_res'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

gpath = IMAGENET.parent/('super_res_gen_128.h5')
#gpath = Path('data/super_res_gen_128.h5')


keep_pct=1.0
bs=32
sz=128
default_sz=400

c_lr=5e-4
c_lrs = np.array([c_lr,c_lr,c_lr,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [5]:
netG = Unet34(nf_factor=1, self_attention=True, sn=True, leakyReLu=False, scale=2).cuda()
#netG = Unet34(nf_factor=1, self_attention=True, sn=True, leakyReLu=False).cpu()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
load_model(netG, gpath)
netG = netG.eval()
#netD = DCCritic(ni=3, nf=32, scale=64, sn=sn, self_attention=True).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

RuntimeError: Error(s) in loading state_dict for Unet34:
	size mismatch for up1.x_conv.seq.0.bias: copying a param of torch.Size([256]) from checkpoint, where the shape is torch.Size([128]) in current model.
	size mismatch for up1.x_conv.seq.0.weight_orig: copying a param of torch.Size([256, 256, 1, 1]) from checkpoint, where the shape is torch.Size([128, 256, 1, 1]) in current model.
	size mismatch for up1.x_conv.seq.0.weight: copying a param of torch.Size([256, 256, 1, 1]) from checkpoint, where the shape is torch.Size([128, 256, 1, 1]) in current model.
	size mismatch for up1.x_conv.seq.0.weight_u: copying a param of torch.Size([256]) from checkpoint, where the shape is torch.Size([128]) in current model.
	size mismatch for up1.tr_conv.sequence.0.0.seq.0.bias: copying a param of torch.Size([1024]) from checkpoint, where the shape is torch.Size([512]) in current model.
	size mismatch for up1.tr_conv.sequence.0.0.seq.0.weight_orig: copying a param of torch.Size([1024, 512, 3, 3]) from

In [ ]:
#x_tfms = [BlackAndWhiteTransform(),CenterCrop(sz=default_sz)]
x_tfms = []
data_loader = ImageGenDataLoader(sz=sz, bs=bs, path=IMAGENET, random_seed=42, x_noise=False,
            keep_pct=keep_pct, x_tfms=x_tfms)

md = data_loader.get_model_data()

In [ ]:
#trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)], citers=1, sn=sn)
#trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
#trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8), clr_gen=(1,8), cycle_len=1, epochs=1)

In [ ]:
vis = ModelImageVisualizer(default_sz=default_sz)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/overmiller.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/einstein_beach.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/abe.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/airmen1943.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/20sWoman.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/egypt-1.jpg", netG, md.val_ds)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Rutherford_Hayes.jpg", netG, md.val_ds, tfms=x_tfms, sz=320)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/einstein_portrait.jpg", netG, md.val_ds, tfms=x_tfms, sz=320)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/pinkerton.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/marilyn_woods.jpg", netG, md.val_ds, tfms=x_tfms, sz=350)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/WaltWhitman.jpg", netG, md.val_ds, tfms=x_tfms, sz=280)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dorothea-lange.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Hemmingway2.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Chief.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/hemmingway.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/smoking_kid.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/teddy_rubble.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dustbowl_2.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/camera_man.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/migrant_mother.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/marktwain.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/HelenKeller.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Evelyn_Nesbit.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Eddie-Adams.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/soldier_kids.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/AnselAdamsYosemite.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/unnamed.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/workers_canyon.jpg", netG, md.val_ds, tfms=x_tfms,sz=600)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/CottonMill.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/JudyGarland.jpeg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/kids_pit.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/last_samurai.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/AnselAdamsWhiteChurch.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/opium.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dorothea_lange_2.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/rgs.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/wh-auden.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/w-b-yeats.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/marilyn_portrait.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/wilson-slaverevivalmeeting.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ww1_trench.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/women-bikers.png", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Unidentified1855.jpg", netG, md.val_ds, tfms=x_tfms, sz=400)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/skycrapper_lunch.jpg", netG, md.val_ds, tfms=x_tfms, sz=550)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/sioux.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/school_kids.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/royal_family.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/redwood_lumberjacks.jpg", netG, md.val_ds, tfms=x_tfms, sz=550)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/poverty.jpg", netG, md.val_ds, tfms=x_tfms,sz=550)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/paperboy.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/NativeAmericans.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/helmut_newton-.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Greece1911.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FatMenClub.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/EgyptColosus.jpg", netG, md.val_ds, tfms=x_tfms, sz=400)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/egypt-2.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dustbowl_sd.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dustbowl_people.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dustbowl_5.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/dustbowl_1.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/DriveThroughGiantTree.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/covered-wagons-traveling.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/civil-war_2.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/civil_war_4.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/civil_war_3.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/civil_war.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/BritishSlum.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/bicycles.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/brooklyn_girls_1940s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/40sCouple.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1946Wedding.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Dolores1920s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/TitanicGym.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FrenchVillage1950s.jpg", netG, md.val_ds, tfms=x_tfms, sz=520)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ClassDivide1930sBrittain.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1870sSphinx.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890Surfer.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/TV1930s.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1864UnionSoldier.jpg", netG, md.val_ds, tfms=x_tfms, sz=510)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890sMedStudents.png", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/BellyLaughWWI.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/PiggyBackRide.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/HealingTree.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ManPile.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1910Bike.jpg", netG, md.val_ds, tfms=x_tfms, sz=450)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/FreeportIL.jpg", netG, md.val_ds, tfms=x_tfms, sz=400)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/DutchBabyCoupleEllis.jpg", netG, md.val_ds, tfms=x_tfms, sz=380)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/InuitWoman1903.png", netG, md.val_ds, tfms=x_tfms, sz=460)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1920sDancing.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/AirmanDad.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1910Racket.png", netG, md.val_ds, tfms=x_tfms, sz=540)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1880Paris.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Deadwood1860s.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1860sSamauris.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/LondonUnderground1860.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Mid1800sSisters.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1860Girls.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/SanFran1851.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Kabuki1870s.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Mormons1870s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/EgyptianWomenLate1800s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/PicadillyLate1800s.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/SutroBaths1880s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1880sBrooklynBridge.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ChinaOpiumc1880.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Locomotive1880s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ViennaBoys1880s.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/VictorianDragQueen1880s.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Sami1880s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ArkansasCowboys1880s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Ballet1890Russia.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Rottindean1890s.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890sPingPong.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/London1937.png", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Harlem1932.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/OregonTrail1870s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/EasterNyc1911.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1899NycBlizzard.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1916Sweeden.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Edinburgh1920s.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890sShoeShopOhio.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890sTouristsEgypt.png", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1938Reading.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1850Geography.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1901Electrophone.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Texas1938Woman.png", netG, md.val_ds, tfms=x_tfms, sz=520)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/MaioreWoman1895NZ.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/WestVirginiaHouse.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1920sGuadalope.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1909Chicago.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1920sFarmKid.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/ParisLate1800s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1900sDaytonaBeach.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1930sGeorgia.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/NorwegianBride1920s.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Depression.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1888Slum.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/LivingRoom1920Sweeden.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1896NewsBoyGirl.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/PetDucks1927.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1899SodaFountain.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/TimesSquare1955.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/PuppyGify.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890CliffHouseSF.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1908FamilyPhoto.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1900sSaloon.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1890BostonHospital.jpg", netG, md.val_ds, tfms=x_tfms,sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1870Girl.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/AustriaHungaryWomen1890s.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Shack.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/Apsaroke1908.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1948CarsGrandma.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/PlanesManhattan1931.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/WorriedKid1940sNyc.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1920sFamilyPhoto.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/CatWash1931.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1940sBeerRiver.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/VictorianLivingRoom.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1897BlindmansBluff.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1874Mexico.png", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/MadisonSquare1900.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1867MusicianConstantinople.jpg", netG, md.val_ds, tfms=x_tfms, sz=500)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1925Girl.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1907Cowboys.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/WWIIPeeps.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/BabyBigBoots.jpg", netG, md.val_ds, tfms=x_tfms)

In [ ]:
vis.plot_transformed_image("data/uploaded_images/1895BikeMaidens.jpg", netG, md.val_ds, tfms=x_tfms)